In [1]:
import lightgbm as lgbm
import pandas as pd
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [13]:
df_features = pd.read_csv('./all_features_GTZAN.csv', index_col=False)

In [14]:
df_features

,spectral_centroid_max,spectral_centroid_min,spectral_centroid_mean,spectral_centroid_std,spectral_centroid_kurtosis,spectral_centroid_skew,spectral_rolloff_max,spectral_rolloff_min,spectral_rolloff_mean,spectral_rolloff_std,...,mfcc_11_kurtosis,mfcc_11_skew,mfcc_12_max,mfcc_12_min,mfcc_12_mean,mfcc_12_std,mfcc_12_kurtosis,mfcc_12_skew,tempo,label
0,5514.393296,1789.366583,3177.044739,550.514460,1.276833,0.851272,9905.273438,4005.175781,6235.752497,1316.817640,...,-0.381935,-0.087353,20.820358,-32.647717,-4.106478,8.183018,0.174437,-0.170466,95.703125,metal
1,5626.545444,1763.465470,2850.278392,450.497575,7.135793,1.871643,10314.404297,4155.908203,5927.176533,852.377153,...,-0.121696,0.048562,19.133057,-32.525143,-10.891160,7.585576,0.191163,0.234933,151.999081,metal
2,4137.382271,1196.751465,2147.080162,394.532048,3.597655,1.475513,7751.953125,2842.382812,4376.685986,842.405343,...,-0.231261,-0.070840,11.926428,-40.928543,-14.242151,7.794230,0.155817,-0.128623,112.347147,metal
3,4141.859853,1554.571724,2700.604011,411.344742,-0.256947,0.124514,7708.886719,3531.445312,5177.960956,678.921135,...,0.151427,-0.124628,16.922087,-34.641380,-13.000867,6.993345,0.378246,0.133764,99.384014,metal
4,6472.152579,2069.514147,3357.388867,548.511977,1.316818,0.691419,10056.005859,4198.974609,6810.287667,964.515301,...,-0.134237,0.061878,23.652538,-27.224190,-3.430558,7.447456,0.032957,0.119359,135.999178,metal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,4391.659556,845.457549,1892.933025,539.708781,2.105745,1.212417,7945.751953,1464.257812,4043.096202,1151.012678,...,0.817056,-0.391232,22.351257,-40.868973,-11.328872,7.907563,0.470418,-0.159860,103.359375,rock
995,3711.343427,722.342964,1920.394382,433.842722,1.424099,0.381397,7385.888672,1313.525391,3969.786720,1049.250472,...,0.056944,-0.216273,9.922584,-37.322617,-15.724136,6.684946,0.098052,0.070556,117.453835,rock
996,4295.291893,746.216989,2016.695292,484.391038,3.688027,1.362617,7536.621094,1442.724609,3937.178822,910.120302,...,0.008158,0.037291,5.263572,-37.484295,-16.852932,7.445485,-0.325994,0.149632,129.199219,rock
997,4151.205944,463.391600,1314.906896,495.556800,3.614127,1.452516,7579.687500,452.197266,2800.149090,1206.406750,...,-0.247247,-0.136189,14.255430,-38.359097,-9.841519,8.071631,-0.000298,-0.146827,112.347147,rock


In [15]:
X = df_features.drop(['label'], axis=1).values

In [16]:
y = df_features['label'].values

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# Regressão Logística

In [24]:
params = {
    'cls__penalty': ['l2', 'none'],
    'cls__C': [0.5, 1, 2, 5],
    'cls__max_iter': (500, 1000)
}

In [25]:
pipeline_logistic_regression = Pipeline([
    ('scale', StandardScaler()),
    ('var_tresh', VarianceThreshold(threshold=(.8 * (1 - .8)))),
    ('feature_selection', SelectFromModel(lgbm.LGBMClassifier())),
    ('cls', LogisticRegression())
])

In [26]:
grid_logistic_regression = GridSearchCV(pipeline_logistic_regression, params, scoring='accuracy', n_jobs=6, cv=5)

In [27]:
grid_logistic_regression.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scale', StandardScaler()),
                                       ('var_tresh',
                                        VarianceThreshold(threshold=0.15999999999999998)),
                                       ('feature_selection',
                                        SelectFromModel(estimator=LGBMClassifier())),
                                       ('cls', LogisticRegression())]),
             n_jobs=6,
             param_grid={'cls__C': [0.5, 1, 2, 5], 'cls__max_iter': (500, 1000),
                         'cls__penalty': ['l2', 'none']},
             scoring='accuracy')

In [28]:
predictions = grid_logistic_regression.predict(X_test)

In [29]:
print(f"Melhor score no conjunto de validação (acurácia) = {grid_logistic_regression.best_score_}")
print(f"Melhor score no conjunto de teste (acurácia) = {accuracy_score(y_test, predictions)}")

Melhor score no conjunto de validação (acurácia) = 0.7156062639821028
Melhor score no conjunto de teste (acurácia) = 0.792
